In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains
import time
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
import selenium
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
import undetected_chromedriver as uc
import subprocess
import requests
import re
import numpy as np
import requests
import datetime

In [3]:
from sqlalchemy import create_engine
import urllib

In [24]:
quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=DividendPolicyResearch;Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
result = engine.execute("SELECT [Ticker] FROM [dbo].[tb_sp100_tickers]")
#result = engine.execute("SELECT [Ticker] FROM [dbo].[tb_sp100_tickers] where left([Ticker],1) not in ('A','B','C','D','E','F','G','I','H','J','K') and [Ticker] not in ('LIN','LLY','LMT')")

In [25]:
df_SP100_companies = pd.DataFrame(result.fetchall())
tickers_list = df_SP100_companies.iloc[:, 0].tolist()
tickers_list

['AAPL',
 'ABBV',
 'ABT',
 'ACN',
 'ADBE',
 'AIG',
 'AMD',
 'AMGN',
 'AMT',
 'AMZN',
 'AVGO',
 'AXP',
 'BA',
 'BAC',
 'BK',
 'BKNG',
 'BLK',
 'BMY',
 'BRK-B',
 'C',
 'CAT',
 'CHTR',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CRM',
 'CSCO',
 'CVS',
 'CVX',
 'DE',
 'DHR',
 'DIS',
 'DOW',
 'DUK',
 'EMR',
 'EXC',
 'F',
 'FDX',
 'GD',
 'GE',
 'GILD',
 'GM',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KHC',
 'KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [26]:
print(len(tickers_list))

101


In [27]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
driver = uc.Chrome(options=options)
driver.maximize_window()

for ticker in tickers_list:
    ticker = ticker.replace("-",".")
    print("Тікер: " + ticker)

    wait = WebDriverWait(driver, 20)
    urlnm = 'https://stockanalysis.com/stocks/'+ticker+'/company/'
    driver.get(urlnm)
    driver.implicitly_wait(1)
      
    cpnm = driver.find_element(By.CSS_SELECTOR, "div.text-center:nth-child(1)").text
    cpnm = cpnm.lower()
    print("Назва компанії: " + cpnm)
    
    cpnm = cpnm.split()
    if cpnm[0].replace(".","").replace(",","") == 'the':
        cpnm = cpnm[1].replace(".","").replace(",","")
    else:
        cpnm = cpnm[0].replace(".","").replace(",","")
    cpnm = cpnm.replace("'","")
    print("Скорочена назва компанії: " + cpnm)
    
    url_income_statement = 'https://www.macrotrends.net/stocks/charts/'+ticker+'/'+cpnm+'/income-statement?freq=A'
    print("Посилання: " + url_income_statement)
    time.sleep(2)
    
    #driver = webdriver.Chrome()
    #driver.maximize_window()
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 1
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)

    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 1): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 1): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 1
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    driver.implicitly_wait(2)
    
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 2
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)
    driver.execute_script("window.scrollBy(0, 600);")
    scrollbar = driver.find_element(By.ID, 'jqxScrollThumbhorizontalScrollBarjqxgrid')

    time.sleep(1)

    actions = ActionChains(driver)
    time.sleep(1)

    for i in range(1, 6):
        actions.drag_and_drop_by_offset(scrollbar, i * 70, 0).perform()
    time.sleep(1)
    
    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 2): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 2): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 2
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    print("")

Тікер: AAPL
Назва компанії: apple inc
Скорочена назва компанії: apple
Посилання: https://www.macrotrends.net/stocks/charts/AAPL/apple/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: ABBV
Назва компанії: abbvie inc.
Скорочена назва компанії: abbvie
Посилання: https://www.macrotrends.net/stocks/charts/ABBV/abbvie/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: ABT
Назва компанії: abbott laboratories
Скорочена назва компанії: abbott
Посилання: https://www.macrotrends.net/stocks/charts/ABT/abbott/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: ACN
Назва компанії: accenture plc
Скорочена назва компанії: accenture
Посилання: https://www.macrotrends.net/stocks/charts/ACN/accenture/income-statement?fre

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="wrapperjqxgrid"]"}
  (Session info: chrome=123.0.6312.58); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00E74CE3+225091]
	(No symbol) [0x00DA4E31]
	(No symbol) [0x00C49A7A]
	(No symbol) [0x00C8175B]
	(No symbol) [0x00C8188B]
	(No symbol) [0x00CB7882]
	(No symbol) [0x00C9F5A4]
	(No symbol) [0x00CB5CB0]
	(No symbol) [0x00C9F2F6]
	(No symbol) [0x00C779B9]
	(No symbol) [0x00C7879D]
	sqlite3_dbdata_init [0x012E9A83+4064547]
	sqlite3_dbdata_init [0x012F108A+4094762]
	sqlite3_dbdata_init [0x012EB988+4072488]
	sqlite3_dbdata_init [0x00FEC9E9+930953]
	(No symbol) [0x00DB0804]
	(No symbol) [0x00DAAD28]
	(No symbol) [0x00DAAE51]
	(No symbol) [0x00D9CAC0]
	BaseThreadInitThunk [0x774462C4+36]
	RtlSubscribeWnfStateChangeNotification [0x77DA0FA9+1081]
	

## CAPTCHA - перезапуск 

In [28]:
quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=DividendPolicyResearch;Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
result = engine.execute("SELECT [Ticker] FROM [dbo].[tb_sp100_tickers] where [Ticker] not in ('AAPL','ABBV','ABT','ACN','ADBE','AIG','AMD')")

In [31]:
df_SP100_companies = pd.DataFrame(result.fetchall())
tickers_list = df_SP100_companies.iloc[:, 0].tolist()
tickers_list

['AMGN',
 'AMT',
 'AMZN',
 'AVGO',
 'AXP',
 'BA',
 'BAC',
 'BK',
 'BKNG',
 'BLK',
 'BMY',
 'BRK-B',
 'C',
 'CAT',
 'CHTR',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CRM',
 'CSCO',
 'CVS',
 'CVX',
 'DE',
 'DHR',
 'DIS',
 'DOW',
 'DUK',
 'EMR',
 'EXC',
 'F',
 'FDX',
 'GD',
 'GE',
 'GILD',
 'GM',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KHC',
 'KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [32]:
print(len(tickers_list))

94


In [33]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
driver = uc.Chrome(options=options)
driver.maximize_window()

for ticker in tickers_list:
    ticker = ticker.replace("-",".")
    print("Тікер: " + ticker)

    wait = WebDriverWait(driver, 20)
    urlnm = 'https://stockanalysis.com/stocks/'+ticker+'/company/'
    driver.get(urlnm)
    driver.implicitly_wait(1)
      
    cpnm = driver.find_element(By.CSS_SELECTOR, "div.text-center:nth-child(1)").text
    cpnm = cpnm.lower()
    print("Назва компанії: " + cpnm)
    
    cpnm = cpnm.split()
    if cpnm[0].replace(".","").replace(",","") == 'the':
        cpnm = cpnm[1].replace(".","").replace(",","")
    else:
        cpnm = cpnm[0].replace(".","").replace(",","")
    cpnm = cpnm.replace("'","")
    print("Скорочена назва компанії: " + cpnm)
    
    url_income_statement = 'https://www.macrotrends.net/stocks/charts/'+ticker+'/'+cpnm+'/income-statement?freq=A'
    print("Посилання: " + url_income_statement)
    time.sleep(2)
    
    #driver = webdriver.Chrome()
    #driver.maximize_window()
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 1
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)

    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 1): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 1): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 1
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    driver.implicitly_wait(2)
    
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 2
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)
    driver.execute_script("window.scrollBy(0, 600);")
    scrollbar = driver.find_element(By.ID, 'jqxScrollThumbhorizontalScrollBarjqxgrid')

    time.sleep(1)

    actions = ActionChains(driver)
    time.sleep(1)

    for i in range(1, 6):
        actions.drag_and_drop_by_offset(scrollbar, i * 70, 0).perform()
    time.sleep(1)
    
    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 2): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 2): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 2
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    print("")

Тікер: AMGN
Назва компанії: amgen inc.
Скорочена назва компанії: amgen
Посилання: https://www.macrotrends.net/stocks/charts/AMGN/amgen/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: AMT
Назва компанії: american tower corporation
Скорочена назва компанії: american
Посилання: https://www.macrotrends.net/stocks/charts/AMT/american/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: AMZN
Назва компанії: amazon.com, inc.
Скорочена назва компанії: amazoncom
Посилання: https://www.macrotrends.net/stocks/charts/AMZN/amazoncom/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: AVGO
Назва компанії: broadcom inc.
Скорочена назва компанії: broadcom
Посилання: https://www.macrotrends.net/stocks/charts/AVGO/broadc

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="wrapperjqxgrid"]"}
  (Session info: chrome=123.0.6312.58); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00E74CE3+225091]
	(No symbol) [0x00DA4E31]
	(No symbol) [0x00C49A7A]
	(No symbol) [0x00C8175B]
	(No symbol) [0x00C8188B]
	(No symbol) [0x00CB7882]
	(No symbol) [0x00C9F5A4]
	(No symbol) [0x00CB5CB0]
	(No symbol) [0x00C9F2F6]
	(No symbol) [0x00C779B9]
	(No symbol) [0x00C7879D]
	sqlite3_dbdata_init [0x012E9A83+4064547]
	sqlite3_dbdata_init [0x012F108A+4094762]
	sqlite3_dbdata_init [0x012EB988+4072488]
	sqlite3_dbdata_init [0x00FEC9E9+930953]
	(No symbol) [0x00DB0804]
	(No symbol) [0x00DAAD28]
	(No symbol) [0x00DAAE51]
	(No symbol) [0x00D9CAC0]
	BaseThreadInitThunk [0x774462C4+36]
	RtlSubscribeWnfStateChangeNotification [0x77DA0FA9+1081]
	

## CAPTCHA - перезапуск 

In [38]:
quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=DividendPolicyResearch;Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
result = engine.execute("SELECT [Ticker] FROM [dbo].[tb_sp100_tickers] where [Ticker] not in ('AAPL','ABBV','ABT','ACN','ADBE','AIG','AMD','AMGN','AMT', 'AMZN', 'AVGO', 'AXP', 'BA', 'BAC', 'BK', 'BKNG')")

In [39]:
df_SP100_companies = pd.DataFrame(result.fetchall())
tickers_list = df_SP100_companies.iloc[:, 0].tolist()
tickers_list

['BLK',
 'BMY',
 'BRK-B',
 'C',
 'CAT',
 'CHTR',
 'CL',
 'CMCSA',
 'COF',
 'COP',
 'COST',
 'CRM',
 'CSCO',
 'CVS',
 'CVX',
 'DE',
 'DHR',
 'DIS',
 'DOW',
 'DUK',
 'EMR',
 'EXC',
 'F',
 'FDX',
 'GD',
 'GE',
 'GILD',
 'GM',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KHC',
 'KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [40]:
print(len(tickers_list))

85


In [41]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
driver = uc.Chrome(options=options)
driver.maximize_window()

for ticker in tickers_list:
    ticker = ticker.replace("-",".")
    print("Тікер: " + ticker)

    wait = WebDriverWait(driver, 20)
    urlnm = 'https://stockanalysis.com/stocks/'+ticker+'/company/'
    driver.get(urlnm)
    driver.implicitly_wait(1)
      
    cpnm = driver.find_element(By.CSS_SELECTOR, "div.text-center:nth-child(1)").text
    cpnm = cpnm.lower()
    print("Назва компанії: " + cpnm)
    
    cpnm = cpnm.split()
    if cpnm[0].replace(".","").replace(",","") == 'the':
        cpnm = cpnm[1].replace(".","").replace(",","")
    else:
        cpnm = cpnm[0].replace(".","").replace(",","")
    cpnm = cpnm.replace("'","")
    print("Скорочена назва компанії: " + cpnm)
    
    url_income_statement = 'https://www.macrotrends.net/stocks/charts/'+ticker+'/'+cpnm+'/income-statement?freq=A'
    print("Посилання: " + url_income_statement)
    time.sleep(2)
    
    #driver = webdriver.Chrome()
    #driver.maximize_window()
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 1
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)

    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 1): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 1): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 1
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    driver.implicitly_wait(2)
    
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 2
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)
    driver.execute_script("window.scrollBy(0, 600);")
    scrollbar = driver.find_element(By.ID, 'jqxScrollThumbhorizontalScrollBarjqxgrid')

    time.sleep(1)

    actions = ActionChains(driver)
    time.sleep(1)

    for i in range(1, 6):
        actions.drag_and_drop_by_offset(scrollbar, i * 70, 0).perform()
    time.sleep(1)
    
    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 2): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 2): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 2
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    print("")

Тікер: BLK
Назва компанії: blackrock, inc.
Скорочена назва компанії: blackrock
Посилання: https://www.macrotrends.net/stocks/charts/BLK/blackrock/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: BMY
Назва компанії: bristol-myers squibb company
Скорочена назва компанії: bristol-myers
Посилання: https://www.macrotrends.net/stocks/charts/BMY/bristol-myers/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: BRK.B
Назва компанії: berkshire hathaway inc.
Скорочена назва компанії: berkshire
Посилання: https://www.macrotrends.net/stocks/charts/BRK.B/berkshire/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: C
Назва компанії: citigroup inc.
Скорочена назва компанії: citigroup
Посилання: https://www.macrotrend

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="wrapperjqxgrid"]"}
  (Session info: chrome=123.0.6312.58); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00E74CE3+225091]
	(No symbol) [0x00DA4E31]
	(No symbol) [0x00C49A7A]
	(No symbol) [0x00C8175B]
	(No symbol) [0x00C8188B]
	(No symbol) [0x00CB7882]
	(No symbol) [0x00C9F5A4]
	(No symbol) [0x00CB5CB0]
	(No symbol) [0x00C9F2F6]
	(No symbol) [0x00C779B9]
	(No symbol) [0x00C7879D]
	sqlite3_dbdata_init [0x012E9A83+4064547]
	sqlite3_dbdata_init [0x012F108A+4094762]
	sqlite3_dbdata_init [0x012EB988+4072488]
	sqlite3_dbdata_init [0x00FEC9E9+930953]
	(No symbol) [0x00DB0804]
	(No symbol) [0x00DAAD28]
	(No symbol) [0x00DAAE51]
	(No symbol) [0x00D9CAC0]
	BaseThreadInitThunk [0x774462C4+36]
	RtlSubscribeWnfStateChangeNotification [0x77DA0FA9+1081]
	

## CAPTCHA - перезапуск 

In [42]:
quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=DividendPolicyResearch;Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
result = engine.execute("SELECT [Ticker] FROM [dbo].[tb_sp100_tickers] where [Ticker] not in ('AAPL','ABBV','ABT','ACN','ADBE','AIG','AMD','AMGN','AMT', 'AMZN', 'AVGO', 'AXP', 'BA', 'BAC', 'BK', 'BKNG', 'BLK','BMY', 'BRK-B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO','CVS', 'CVX', 'DE', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FDX', 'GD','GE')")

In [43]:
df_SP100_companies = pd.DataFrame(result.fetchall())
tickers_list = df_SP100_companies.iloc[:, 0].tolist()
tickers_list

['GILD',
 'GM',
 'GOOG',
 'GOOGL',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KHC',
 'KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [44]:
print(len(tickers_list))

59


In [46]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
driver = uc.Chrome(options=options)
driver.maximize_window()

for ticker in tickers_list:
    ticker = ticker.replace("-",".")
    print("Тікер: " + ticker)

    wait = WebDriverWait(driver, 20)
    urlnm = 'https://stockanalysis.com/stocks/'+ticker+'/company/'
    driver.get(urlnm)
    driver.implicitly_wait(1)
      
    cpnm = driver.find_element(By.CSS_SELECTOR, "div.text-center:nth-child(1)").text
    cpnm = cpnm.lower()
    print("Назва компанії: " + cpnm)
    
    cpnm = cpnm.split()
    if cpnm[0].replace(".","").replace(",","") == 'the':
        cpnm = cpnm[1].replace(".","").replace(",","")
    else:
        cpnm = cpnm[0].replace(".","").replace(",","")
    cpnm = cpnm.replace("'","").replace("&","")
    print("Скорочена назва компанії: " + cpnm)
    
    url_income_statement = 'https://www.macrotrends.net/stocks/charts/'+ticker+'/'+cpnm+'/income-statement?freq=A'
    print("Посилання: " + url_income_statement)
    time.sleep(2)
    
    #driver = webdriver.Chrome()
    #driver.maximize_window()
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 1
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)

    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 1): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 1): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 1
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    driver.implicitly_wait(2)
    
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 2
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)
    driver.execute_script("window.scrollBy(0, 600);")
    scrollbar = driver.find_element(By.ID, 'jqxScrollThumbhorizontalScrollBarjqxgrid')

    time.sleep(1)

    actions = ActionChains(driver)
    time.sleep(1)

    for i in range(1, 6):
        actions.drag_and_drop_by_offset(scrollbar, i * 70, 0).perform()
    time.sleep(1)
    
    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 2): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 2): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 2
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    print("")

Тікер: GILD
Назва компанії: gilead sciences, inc.
Скорочена назва компанії: gilead
Посилання: https://www.macrotrends.net/stocks/charts/GILD/gilead/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: GM
Назва компанії: general motors company
Скорочена назва компанії: general
Посилання: https://www.macrotrends.net/stocks/charts/GM/general/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: GOOG
Назва компанії: alphabet inc.
Скорочена назва компанії: alphabet
Посилання: https://www.macrotrends.net/stocks/charts/GOOG/alphabet/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: GOOGL
Назва компанії: alphabet inc.
Скорочена назва компанії: alphabet
Посилання: https://www.macrotrends.net/stocks/charts/GOOGL/alph

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="wrapperjqxgrid"]"}
  (Session info: chrome=123.0.6312.58); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00E74CE3+225091]
	(No symbol) [0x00DA4E31]
	(No symbol) [0x00C49A7A]
	(No symbol) [0x00C8175B]
	(No symbol) [0x00C8188B]
	(No symbol) [0x00CB7882]
	(No symbol) [0x00C9F5A4]
	(No symbol) [0x00CB5CB0]
	(No symbol) [0x00C9F2F6]
	(No symbol) [0x00C779B9]
	(No symbol) [0x00C7879D]
	sqlite3_dbdata_init [0x012E9A83+4064547]
	sqlite3_dbdata_init [0x012F108A+4094762]
	sqlite3_dbdata_init [0x012EB988+4072488]
	sqlite3_dbdata_init [0x00FEC9E9+930953]
	(No symbol) [0x00DB0804]
	(No symbol) [0x00DAAD28]
	(No symbol) [0x00DAAE51]
	(No symbol) [0x00D9CAC0]
	BaseThreadInitThunk [0x774462C4+36]
	RtlSubscribeWnfStateChangeNotification [0x77DA0FA9+1081]
	

## CAPTCHA - перезапуск

In [47]:
quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=DividendPolicyResearch;Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
result = engine.execute("SELECT [Ticker] FROM [dbo].[tb_sp100_tickers] where [Ticker] not in ('AAPL','ABBV','ABT','ACN','ADBE','AIG','AMD','AMGN','AMT', 'AMZN', 'AVGO', 'AXP', 'BA', 'BAC', 'BK', 'BKNG', 'BLK','BMY', 'BRK-B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO','CVS', 'CVX', 'DE', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FDX', 'GD','GE','GILD','GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON')")

In [48]:
df_SP100_companies = pd.DataFrame(result.fetchall())
tickers_list = df_SP100_companies.iloc[:, 0].tolist()
tickers_list

['IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KHC',
 'KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [49]:
print(len(tickers_list))

52


In [50]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
driver = uc.Chrome(options=options)
driver.maximize_window()

for ticker in tickers_list:
    ticker = ticker.replace("-",".")
    print("Тікер: " + ticker)

    wait = WebDriverWait(driver, 20)
    urlnm = 'https://stockanalysis.com/stocks/'+ticker+'/company/'
    driver.get(urlnm)
    driver.implicitly_wait(1)
      
    cpnm = driver.find_element(By.CSS_SELECTOR, "div.text-center:nth-child(1)").text
    cpnm = cpnm.lower()
    print("Назва компанії: " + cpnm)
    
    cpnm = cpnm.split()
    if cpnm[0].replace(".","").replace(",","") == 'the':
        cpnm = cpnm[1].replace(".","").replace(",","")
    else:
        cpnm = cpnm[0].replace(".","").replace(",","")
    cpnm = cpnm.replace("'","").replace("&","")
    print("Скорочена назва компанії: " + cpnm)
    
    url_income_statement = 'https://www.macrotrends.net/stocks/charts/'+ticker+'/'+cpnm+'/income-statement?freq=A'
    print("Посилання: " + url_income_statement)
    time.sleep(2)
    
    #driver = webdriver.Chrome()
    #driver.maximize_window()
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 1
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)

    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 1): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 1): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 1
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    driver.implicitly_wait(2)
    
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 2
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(1)
    driver.execute_script("window.scrollBy(0, 600);")
    scrollbar = driver.find_element(By.ID, 'jqxScrollThumbhorizontalScrollBarjqxgrid')

    time.sleep(1)

    actions = ActionChains(driver)
    time.sleep(1)

    for i in range(1, 6):
        actions.drag_and_drop_by_offset(scrollbar, i * 70, 0).perform()
    time.sleep(1)
    
    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    divList = soup.findAll('div', {'role': 'row'})
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 2): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 2): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 2
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    print("")

Тікер: IBM
Назва компанії: international business machines corporation
Скорочена назва компанії: international
Посилання: https://www.macrotrends.net/stocks/charts/IBM/international/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: INTC
Назва компанії: intel corporation
Скорочена назва компанії: intel
Посилання: https://www.macrotrends.net/stocks/charts/INTC/intel/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: JNJ
Назва компанії: johnson & johnson
Скорочена назва компанії: johnson
Посилання: https://www.macrotrends.net/stocks/charts/JNJ/johnson/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: JPM
Назва компанії: jpmorgan chase & co.
Скорочена назва компанії: jpmorgan
Посилання: https://www.macrot

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="wrapperjqxgrid"]"}
  (Session info: chrome=123.0.6312.58); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00E74CE3+225091]
	(No symbol) [0x00DA4E31]
	(No symbol) [0x00C49A7A]
	(No symbol) [0x00C8175B]
	(No symbol) [0x00C8188B]
	(No symbol) [0x00CB7882]
	(No symbol) [0x00C9F5A4]
	(No symbol) [0x00CB5CB0]
	(No symbol) [0x00C9F2F6]
	(No symbol) [0x00C779B9]
	(No symbol) [0x00C7879D]
	sqlite3_dbdata_init [0x012E9A83+4064547]
	sqlite3_dbdata_init [0x012F108A+4094762]
	sqlite3_dbdata_init [0x012EB988+4072488]
	sqlite3_dbdata_init [0x00FEC9E9+930953]
	(No symbol) [0x00DB0804]
	(No symbol) [0x00DAAD28]
	(No symbol) [0x00DAAE51]
	(No symbol) [0x00D9CAC0]
	BaseThreadInitThunk [0x774462C4+36]
	RtlSubscribeWnfStateChangeNotification [0x77DA0FA9+1081]
	

## CAPTCHA - перезапуск із збільшенням часових інтервалів між діями

In [51]:
quoted = urllib.parse.quote_plus("Driver={SQL Server Native Client 11.0};Server=VIKTOR-PC-2001;Database=DividendPolicyResearch;Trusted_Connection=yes;")
engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
result = engine.execute("SELECT [Ticker] FROM [dbo].[tb_sp100_tickers] where [Ticker] not in ('AAPL','ABBV','ABT','ACN','ADBE','AIG','AMD','AMGN','AMT', 'AMZN', 'AVGO', 'AXP', 'BA', 'BAC', 'BK', 'BKNG', 'BLK','BMY', 'BRK-B', 'C', 'CAT', 'CHTR', 'CL', 'CMCSA', 'COF', 'COP', 'COST', 'CRM', 'CSCO','CVS', 'CVX', 'DE', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FDX', 'GD','GE','GILD','GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM','INTC', 'JNJ', 'JPM', 'KHC')")

In [52]:
df_SP100_companies = pd.DataFrame(result.fetchall())
tickers_list = df_SP100_companies.iloc[:, 0].tolist()
tickers_list

['KO',
 'LIN',
 'LLY',
 'LMT',
 'LOW',
 'MA',
 'MCD',
 'MDLZ',
 'MDT',
 'MET',
 'META',
 'MMM',
 'MO',
 'MRK',
 'MS',
 'MSFT',
 'NEE',
 'NFLX',
 'NKE',
 'NVDA',
 'ORCL',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RTX',
 'SBUX',
 'SCHW',
 'SO',
 'SPG',
 'T',
 'TGT',
 'TMO',
 'TMUS',
 'TSLA',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB',
 'V',
 'VZ',
 'WFC',
 'WMT',
 'XOM']

In [53]:
print(len(tickers_list))

47


In [54]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
driver = uc.Chrome(options=options)
driver.maximize_window()

for ticker in tickers_list:
    ticker = ticker.replace("-",".")
    print("Тікер: " + ticker)

    wait = WebDriverWait(driver, 20)
    urlnm = 'https://stockanalysis.com/stocks/'+ticker+'/company/'
    driver.get(urlnm)
    driver.implicitly_wait(1)
      
    cpnm = driver.find_element(By.CSS_SELECTOR, "div.text-center:nth-child(1)").text
    cpnm = cpnm.lower()
    print("Назва компанії: " + cpnm)
    
    cpnm = cpnm.split()
    if cpnm[0].replace(".","").replace(",","") == 'the':
        cpnm = cpnm[1].replace(".","").replace(",","")
    else:
        cpnm = cpnm[0].replace(".","").replace(",","")
    cpnm = cpnm.replace("'","").replace("&","")
    print("Скорочена назва компанії: " + cpnm)
    
    url_income_statement = 'https://www.macrotrends.net/stocks/charts/'+ticker+'/'+cpnm+'/income-statement?freq=A'
    print("Посилання: " + url_income_statement)
    time.sleep(3)
    
    #driver = webdriver.Chrome()
    #driver.maximize_window()
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 1
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(3)

    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    driver.implicitly_wait(2)
    divList = soup.findAll('div', {'role': 'row'})
    driver.implicitly_wait(1)
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 1): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 1): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 1
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    driver.implicitly_wait(2)
    
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 2
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(3)
    driver.execute_script("window.scrollBy(0, 600);")
    scrollbar = driver.find_element(By.ID, 'jqxScrollThumbhorizontalScrollBarjqxgrid')

    time.sleep(2)

    actions = ActionChains(driver)
    time.sleep(2)

    for i in range(1, 6):
        actions.drag_and_drop_by_offset(scrollbar, i * 70, 0).perform()
    time.sleep(1)
    
    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    driver.implicitly_wait(2)
    divList = soup.findAll('div', {'role': 'row'})
    driver.implicitly_wait(1)
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 2): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 2): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 2
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    print("")

Тікер: KO
Назва компанії: the coca-cola company
Скорочена назва компанії: coca-cola
Посилання: https://www.macrotrends.net/stocks/charts/KO/coca-cola/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: LIN
Назва компанії: linde plc
Скорочена назва компанії: linde
Посилання: https://www.macrotrends.net/stocks/charts/LIN/linde/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: LLY
Назва компанії: eli lilly and company
Скорочена назва компанії: eli
Посилання: https://www.macrotrends.net/stocks/charts/LLY/eli/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

Тікер: LMT
Назва компанії: lockheed martin corporation
Скорочена назва компанії: lockheed
Посилання: https://www.macrotrends.net/stocks/charts/LMT/lockheed/in

# Дві компанії, для яких не спрацював автоматичний скролінг (через затримки / випливаючі вікна) - перезапуск

In [55]:
tickers_list = ['LOW']

In [57]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
driver = uc.Chrome(options=options)
driver.maximize_window()

for ticker in tickers_list:
    ticker = ticker.replace("-",".")
    print("Тікер: " + ticker)

    wait = WebDriverWait(driver, 20)
    urlnm = 'https://stockanalysis.com/stocks/'+ticker+'/company/'
    driver.get(urlnm)
    driver.implicitly_wait(1)
      
    cpnm = driver.find_element(By.CSS_SELECTOR, "div.text-center:nth-child(1)").text
    cpnm = cpnm.lower()
    print("Назва компанії: " + cpnm)
    
    cpnm = cpnm.split()
    if cpnm[0].replace(".","").replace(",","") == 'the':
        cpnm = cpnm[1].replace(".","").replace(",","")
    else:
        cpnm = cpnm[0].replace(".","").replace(",","")
    cpnm = cpnm.replace("'","").replace("&","")
    print("Скорочена назва компанії: " + cpnm)
    
    url_income_statement = 'https://www.macrotrends.net/stocks/charts/'+ticker+'/'+cpnm+'/income-statement?freq=A'
    print("Посилання: " + url_income_statement)
    time.sleep(3)
    
    #driver = webdriver.Chrome()
    #driver.maximize_window()
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 1
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(3)

    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    driver.implicitly_wait(2)
    divList = soup.findAll('div', {'role': 'row'})
    driver.implicitly_wait(1)
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 1): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 1): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 1
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    driver.implicitly_wait(2)
    
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 2
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(3)
    driver.execute_script("window.scrollBy(0, 600);")
    scrollbar = driver.find_element(By.ID, 'jqxScrollThumbhorizontalScrollBarjqxgrid')

    time.sleep(2)

    actions = ActionChains(driver)
    time.sleep(2)

    for i in range(1, 6):
        actions.drag_and_drop_by_offset(scrollbar, i * 70, 0).perform()
    time.sleep(1)
    
    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    driver.implicitly_wait(2)
    divList = soup.findAll('div', {'role': 'row'})
    driver.implicitly_wait(1)
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 2): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 2): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 2
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    print("")

Тікер: LOW
Назва компанії: lowe's companies, inc.
Скорочена назва компанії: lowes
Посилання: https://www.macrotrends.net/stocks/charts/LOW/lowes/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11



In [58]:
tickers_list = ['HD']

In [60]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
driver = uc.Chrome(options=options)
driver.maximize_window()

for ticker in tickers_list:
    ticker = ticker.replace("-",".")
    print("Тікер: " + ticker)

    wait = WebDriverWait(driver, 20)
    urlnm = 'https://stockanalysis.com/stocks/'+ticker+'/company/'
    driver.get(urlnm)
    driver.implicitly_wait(1)
      
    cpnm = driver.find_element(By.CSS_SELECTOR, "div.text-center:nth-child(1)").text
    cpnm = cpnm.lower()
    print("Назва компанії: " + cpnm)
    
    cpnm = cpnm.split()
    if cpnm[0].replace(".","").replace(",","") == 'the':
        cpnm = cpnm[1].replace(".","").replace(",","")
    else:
        cpnm = cpnm[0].replace(".","").replace(",","")
    cpnm = cpnm.replace("'","").replace("&","")
    print("Скорочена назва компанії: " + cpnm)
    
    url_income_statement = 'https://www.macrotrends.net/stocks/charts/'+ticker+'/'+cpnm+'/income-statement?freq=A'
    print("Посилання: " + url_income_statement)
    time.sleep(3)
    
    #driver = webdriver.Chrome()
    #driver.maximize_window()
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 1
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(3)

    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    driver.implicitly_wait(2)
    divList = soup.findAll('div', {'role': 'row'})
    driver.implicitly_wait(1)
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 1): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 1): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 1
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    driver.implicitly_wait(2)
    
    driver.execute_script(
        f"window.location = '{url_income_statement}';")
    driver.implicitly_wait(4)

    # частина 2
    grid = driver.find_element(By.ID, 'wrapperjqxgrid')
    time.sleep(3)
    driver.execute_script("window.scrollBy(0, 600);")
    scrollbar = driver.find_element(By.ID, 'jqxScrollThumbhorizontalScrollBarjqxgrid')

    time.sleep(2)

    actions = ActionChains(driver)
    time.sleep(2)

    for i in range(1, 6):
        actions.drag_and_drop_by_offset(scrollbar, i * 70, 0).perform()
    time.sleep(1)
    
    soup = BeautifulSoup(grid.get_attribute('outerHTML'), "html.parser")
    headersList = soup.findAll('div', {'role': 'columnheader'})
    col_names=[h.text for h in headersList]
    driver.implicitly_wait(2)
    divList = soup.findAll('div', {'role': 'row'})
    driver.implicitly_wait(1)
    data = [[x.text for x in div.findChildren('div', recursive=False)] for div in divList]
    df = pd.DataFrame(data, columns=col_names)
    print("К-ть стовпців (частина 2): " + str(df.index.size))
    
    df = df.rename(columns={"Annual Data | Millions of US $ except per share data": "Year"})
    df = df.transpose()
    df['index1'] = df.index
    headers = df.iloc[0]
    new_df  = pd.DataFrame(df.values[2:], columns=headers)
    print("К-ть рядків (частина 2): " + str(new_df.index.size))
    new_df['ticker_name'] = ticker.replace(".","-")
    new_df["date_insert"] = datetime.datetime.now().strftime("%Y-%m-%d")
    new_df["time_insert"] = datetime.datetime.now().strftime("%H:%M:%S")
    new_df["period type"] = 2
    #print(new_df)
    
    new_df.to_sql('tb_FS_income_statement_macrotrends', schema='dbo', con = engine, if_exists='append',index=False)
    
    print("")

Тікер: HD
Назва компанії: the home depot, inc.
Скорочена назва компанії: home
Посилання: https://www.macrotrends.net/stocks/charts/HD/home/income-statement?freq=A
К-ть стовпців (частина 1): 22
К-ть рядків (частина 1): 7
К-ть стовпців (частина 2): 22
К-ть рядків (частина 2): 11

